In [2]:
import os
import pandas as pd
import ast

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.ai.documentintelligence.models import AnalyzeResult

from shared.prompts.prompts import df_transform
from shared.transformer.chat_gpt import get_completion
from shared.cloud_storage import CloudStorage
from shared.loaders.bigquery_loader import Loader
from shared.extractors.api.bigquery_extractor import Extractor


endpoint = "https://utadeo.cognitiveservices.azure.com/"
key = os.getenv("AZURE_KEY")

def corregir_comillas(diccionario):
    for clave, valor in diccionario.items():
        if isinstance(valor, str) and "'" in valor:
            diccionario[clave] = valor.replace("'", "")
    return diccionario


def analyze_layout(path_to_sample_documents):
    document_intelligence_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    with open(path_to_sample_documents, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-invoice", document=f, locale="en-US"
        )

    result: AnalyzeResult = poller.result()

    if result.tables:
        all_tables = pd.DataFrame()
        for table_idx, table in enumerate(result.tables):
            print(f"Table # {table_idx} has {table.row_count} rows and {table.column_count} columns")
            if table.row_count > 3:
                table_data = {}
                for cell in table.cells:
                    row_index = cell.row_index
                    column_index = cell.column_index
                    content = cell.content
                    if row_index not in table_data:
                        table_data[row_index] = {}
                    table_data[row_index][column_index] = content

                df = pd.DataFrame.from_dict(table_data, orient="index").sort_index(axis=1)
                df = df.apply(lambda x: x.str.replace('\n', ''))
                df = df.apply(lambda x: x.str.replace("'", ''))
                df = df.apply(lambda x: x.str.replace('"', ''))
                print(df)
                results = get_completion(df_transform.format(dataframe=df))
                print(results)
                data_list = ast.literal_eval(results)
                df_lista = pd.DataFrame(data_list)
                all_tables = pd.concat([all_tables, df_lista], ignore_index=True)
        all_tables = all_tables.applymap(lambda x: x.upper() if isinstance(x, str) else x)
        all_tables['source']=path_to_sample_documents
        return all_tables

def main():
    cloud = CloudStorage()
    lista = cloud.list_files_in_gcs("pdfs_utadeo", "pdfs/")
    print(lista)
    lista.remove("pdfs/")
    df_storage = pd.DataFrame()
    df_storage["file_path"] = lista
    df_storage["flag"] = 0

    Extract = Extractor()
    df_bigquery = Extract.extract_bigquery("artful-sled-419501.secop.pdf_control")
    nuevos_pdfs = df_storage[~df_storage["file_path"].isin(df_bigquery["file_path"])]
    
    load = Loader()
    if not nuevos_pdfs.empty:
        load.load_bigquery_df(nuevos_pdfs, "artful-sled-419501.secop.pdf_control", "WRITE_APPEND")

    df_bigquery_new = Extract.extract_bigquery_flag("artful-sled-419501.secop.pdf_control")
    if not df_bigquery_new.empty:
        for row in df_bigquery_new.itertuples():
            print('processing the file' + row.file_path)
            destino = f"/home/juanstn/{row.file_path.split('/')[-1]}"
            cloud.download_blob("pdfs_utadeo", row.file_path, destino)
            df_to_load = analyze_layout(destino)
    return df_to_load

if __name__ == "__main__":
    df_load = main()
    print(df_load)


['pdfs/', 'pdfs/C_PROCESO_21-4-11507224_22587527_83602193-1-2.pdf', 'pdfs/C_PROCESO_21-4-11747078_225815018_90139203-3-4 (2).pdf']
processing the filepdfs/C_PROCESO_21-4-11747078_225815018_90139203-3-4 (2).pdf
Archivo pdfs/C_PROCESO_21-4-11747078_225815018_90139203-3-4 (2).pdf descargado en /home/juanstn/C_PROCESO_21-4-11747078_225815018_90139203-3-4 (2).pdf.
Table # 0 has 31 rows and 4 columns
                                                    0  \
0                                         DESCRIPCION   
1                               ACEITE MINERAL 500 CC   
2                            ACETAMINOFEN500MGTABLETA   
3                  ACETAMINOFEN100MG/MLGOTAS FCOX30ML   
4                          ACETAMINOFENJARABE150MG/ML   
5                          ACETATO DE ALUMINIO SOBRES   
6                           ACETAZOLAMIDA250MGTABLETA   
7                            ACICLOVIR 200 MG TABLETA   
8                           ACICLOVIR CREMA TOPICA 5%   
9                             AC

/tmp/ipykernel_5089/1718574155.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_tables = all_tables.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [4]:
load = Loader()
load.load_bigquery_df(df_load, "artful-sled-419501.secop.detalle_contratos", "WRITE_APPEND")

Loaded 126 rows and 7 columns to artful-sled-419501.secop.detalle_contratos


In [10]:
import datetime

def create_weeks(initial_date, last_date):
    initial_date = datetime.datetime.strptime(initial_date, "%Y-%m-%d")
    last_date = datetime.datetime.strptime(last_date, "%Y-%m-%d")
    weeks = []
    while initial_date < last_date:
        week_start = initial_date.strftime("%Y-%m-%d")
        week_end = (initial_date + datetime.timedelta(days=6)).strftime("%Y-%m-%d")
        weeks.append({
            "initial_date": week_start + "T00:00:00.000",
            "last_date": week_end + "T00:00:00.000"
        })
        initial_date = initial_date + datetime.timedelta(days=7)
    return weeks


for week in create_weeks("2021-01-01", "2022-01-31"):
    initial_date = week["initial_date"]
    last_date = week["last_date"]
    print(initial_date, last_date)

2021-01-01T00:00:00.000 2021-01-07T00:00:00.000
2021-01-08T00:00:00.000 2021-01-14T00:00:00.000
2021-01-15T00:00:00.000 2021-01-21T00:00:00.000
2021-01-22T00:00:00.000 2021-01-28T00:00:00.000
2021-01-29T00:00:00.000 2021-02-04T00:00:00.000
2021-02-05T00:00:00.000 2021-02-11T00:00:00.000
2021-02-12T00:00:00.000 2021-02-18T00:00:00.000
2021-02-19T00:00:00.000 2021-02-25T00:00:00.000
2021-02-26T00:00:00.000 2021-03-04T00:00:00.000
2021-03-05T00:00:00.000 2021-03-11T00:00:00.000
2021-03-12T00:00:00.000 2021-03-18T00:00:00.000
2021-03-19T00:00:00.000 2021-03-25T00:00:00.000
2021-03-26T00:00:00.000 2021-04-01T00:00:00.000
2021-04-02T00:00:00.000 2021-04-08T00:00:00.000
2021-04-09T00:00:00.000 2021-04-15T00:00:00.000
2021-04-16T00:00:00.000 2021-04-22T00:00:00.000
2021-04-23T00:00:00.000 2021-04-29T00:00:00.000
2021-04-30T00:00:00.000 2021-05-06T00:00:00.000
2021-05-07T00:00:00.000 2021-05-13T00:00:00.000
2021-05-14T00:00:00.000 2021-05-20T00:00:00.000
2021-05-21T00:00:00.000 2021-05-27T00:00